## Exercícios

1. Considere a existência de três tabelas distintas:
* customer.csv : Possui a informação dos clientes em duas colunas: customer id  customer name
* products.csv : Conté informação dos produtos vendidos pela empresa em três colunas - p_id (product id), product (name) e price
* sales.csv : Contém informações das vendas realizadas em seis colunas - sale_id, c_id (customer id), p_id (product_id), qty (quantity sold), store (name)

Conhecendo as bases e utilizando os métodos de concatenação de bases responda:


a) Quais produtos não foram vendidos?

In [1]:
import pandas as pd
import numpy as np

In [2]:
customer = pd.read_csv('./data/customer.csv', sep=',')

In [3]:
products = pd.read_csv('./data/products.csv', sep=',')

In [4]:
sales = pd.read_csv('./data/sales.csv', sep=',')

In [5]:
customer.head()

,c_id,Customer
0,1,Rabi
1,2,Raju
2,3,Alex
3,4,Rani
4,5,King


In [6]:
products.head()

,p_id,product,price
0,1,Hard Disk,80
1,2,RAM,90
2,3,Monitor,75
3,4,CPU,55
4,5,Keyboard,20


In [7]:
sales

,sale_id,c_id,p_id,product,qty,store
0,1,2,3,Monitor,2,ABC
1,2,2,4,CPU,1,DEF
2,3,1,3,Monitor,3,ABC
3,4,4,2,RAM,2,DEF
4,5,2,3,Monitor,3,ABC
5,6,3,3,Monitor,2,DEF
6,7,2,2,RAM,3,ABC
7,8,3,2,RAM,2,DEF
8,9,2,3,Monitor,2,ABC


In [8]:
#resposta no exercício e)

b) Quantos clientes não realizaram uma compra? 

In [9]:
sales.merge(customer, how="outer", on="c_id")['sale_id'].isna().sum()

4

c) Liste a quantidade vendida e o faturamento de cada produto 

In [10]:
sales_total = sales.merge(products, how="left", on=["product"])

In [11]:
sales_total

,sale_id,c_id,p_id_x,product,qty,store,p_id_y,price
0,1,2,3,Monitor,2,ABC,3,75
1,2,2,4,CPU,1,DEF,4,55
2,3,1,3,Monitor,3,ABC,3,75
3,4,4,2,RAM,2,DEF,2,90
4,5,2,3,Monitor,3,ABC,3,75
5,6,3,3,Monitor,2,DEF,3,75
6,7,2,2,RAM,3,ABC,2,90
7,8,3,2,RAM,2,DEF,2,90
8,9,2,3,Monitor,2,ABC,3,75


In [12]:
#usando .groupby()
prod_sales_total = sales_total.groupby(['product', 'price'], as_index=False)[['qty']].sum()

In [13]:
prod_sales_total['revenue'] = prod_sales_total['qty']*prod_sales_total['price']

In [14]:
prod_sales_total

,product,price,qty,revenue
0,CPU,55,1,55
1,Monitor,75,12,900
2,RAM,90,7,630


In [15]:
#usando .pivot_table()
sales_price = pd.pivot_table(sales_total, index=['product', 'price'], values='qty', aggfunc=sum).reset_index()

In [16]:
sales_price['revenue'] = sales_price['qty']*sales_price['price']

In [17]:
sales_price.rename(index={'product_x' :'product'}, inplace=True)
sales_price

,product,price,qty,revenue
0,CPU,55,1,55
1,Monitor,75,12,900
2,RAM,90,7,630


d) Liste a quantidade vendida de cada produto por loja

In [18]:
#usando .pivot_table()
store_prod = sales.pivot_table( index=['store'], values=['qty'], columns='product', aggfunc=sum).fillna(0).reset_index()

In [19]:
store_prod

store  qty             
product        CPU Monitor  RAM
0         ABC  0.0    10.0  3.0
1         DEF  1.0     2.0  4.0

In [21]:
# usando .groupby()
store_prod_gr = sales.groupby(['store', 'product'], as_index=False)['qty'].sum()

In [22]:
store_prod_gr

,store,product,qty
0,ABC,Monitor,10
1,ABC,RAM,3
2,DEF,CPU,1
3,DEF,Monitor,2
4,DEF,RAM,4


e) Qual loja teve maior faturamento?

In [23]:
products

,p_id,product,price
0,1,Hard Disk,80
1,2,RAM,90
2,3,Monitor,75
3,4,CPU,55
4,5,Keyboard,20
5,6,Mouse,10
6,7,Motherboard,50
7,8,Power supply,20


In [24]:
store_revenue = store_prod_gr.merge(products, how='outer', on=['product'])

In [25]:
store_revenue['store_revenue'] = store_revenue['qty']*store_revenue['price']

In [26]:
# produtos que não foram vendidos..resposta a)
store_revenue[(store_revenue['qty'].isna())]

,store,product,qty,p_id,price,store_revenue
5,NaN,Hard Disk,NaN,1,80,NaN
6,NaN,Keyboard,NaN,5,20,NaN
7,NaN,Mouse,NaN,6,10,NaN
8,NaN,Motherboard,NaN,7,50,NaN
9,NaN,Power supply,NaN,8,20,NaN


In [27]:
store_revenue.groupby(['store'])[['store_revenue']].sum()

,store_revenue
store,
ABC,1020.0
DEF,565.0


f) Qual produto foi o mais vendido?

In [28]:
sales.groupby('product',as_index=False )['qty'].sum().sort_values(by='qty',ascending=False)

,product,qty
1,Monitor,12
2,RAM,7
0,CPU,1


In [29]:
sales.pivot_table(index='product', values='qty', aggfunc=sum).sort_values(by='qty', ascending=False).reset_index()

,product,qty
0,Monitor,12
1,RAM,7
2,CPU,1
